In [1]:
import numpy
import cv2
import os
from tqdm import tqdm

In [2]:
path = "../input/dynamic-sign-language-gestures-01/all"
image_size = 256
frame_rate = 0.1

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (image_size, image_size))
    return [hasFrames, image]

In [3]:
videos_list = sorted(os.listdir(path))[:500]
new_list = []

for xyz in tqdm(range(10)):
    mno = xyz*50
    for index in range(mno, mno+50):
        new_list.append(videos_list[index])

100%|██████████| 10/10 [00:00<00:00, 48044.72it/s]


In [4]:
frames = []
labels = []

for video in tqdm(new_list):
#     vidcap = cv2.VideoCapture(path + '/' + video)
#     sec = 0
#     mini = []
#     success, frame = getFrame(sec)
#     while success:
#         mini.append(frame)
#         sec = sec + frame_rate
#         sec = round(sec, 2)
#         success, frame = getFrame(sec)
        
#     frames.append(mini[-8:])
    labels.extend([int(video[:3]) -1]*8)

100%|██████████| 500/500 [00:00<00:00, 315788.59it/s]


In [5]:
from tensorflow.keras.utils import to_categorical
# frames = numpy.array(frames)
frames = numpy.load('../input/aslframes/last8-10classes.npy')
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(frames, cat_labels, test_size=0.1,
                                                   shuffle=True, random_state=42, stratify=labels)

In [12]:
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = VGG16(include_top=False)
embed = GlobalMaxPooling2D()(base_model.output)
layer = Dropout(0.4)(embed)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.4)(layer)
layer = Dense(256, activation='relu')(layer)
layer = Dense(64, activation='relu')(layer)
output = Dense(10, activation='softmax')(layer)

model = Model(inputs=base_model.inputs, outputs=output)

94773248/94765736 [==============================] - 1s 0us/step: 


In [13]:
base_model.trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(filepath='model.h5', monitor='val_accuracy',
                                                       save_best_only=True, verbose=1)
earlyStop = tensorflow.keras.callbacks.EarlyStopping(patience=16, monitor='val_accuracy',
                                                    restore_best_weights=True, verbose=1)
reduceLR = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, 
                                                patience=4, verbose=1)

model.fit(x_train, y_train, batch_size=32, epochs=128, verbose=1,
          validation_data = (x_test, y_test),
          callbacks=[checkpoint, earlyStop, reduceLR])

Epoch 1/128
113/113 [==============================] - 7s 37ms/step - loss: 4.2475 - accuracy: 0.1364 - val_loss: 2.0061 - val_accuracy: 0.2950

Epoch 00001: val_accuracy improved from -inf to 0.29500, saving model to model.h5
Epoch 2/128
113/113 [==============================] - 3s 28ms/step - loss: 2.0224 - accuracy: 0.2565 - val_loss: 1.8690 - val_accuracy: 0.3425

Epoch 00002: val_accuracy improved from 0.29500 to 0.34250, saving model to model.h5
Epoch 3/128
113/113 [==============================] - 3s 28ms/step - loss: 1.7238 - accuracy: 0.3381 - val_loss: 1.2791 - val_accuracy: 0.4925

Epoch 00003: val_accuracy improved from 0.34250 to 0.49250, saving model to model.h5
Epoch 4/128
113/113 [==============================] - 3s 28ms/step - loss: 1.2195 - accuracy: 0.5199 - val_loss: 0.9288 - val_accuracy: 0.6750

Epoch 00004: val_accuracy improved from 0.49250 to 0.67500, saving model to model.h5
Epoch 5/128
113/113 [==============================] - 3s 28ms/step - loss: 0.9916 

113/113 [==============================] - 3s 28ms/step - loss: 0.0751 - accuracy: 0.9769 - val_loss: 0.1146 - val_accuracy: 0.9675

Epoch 00038: val_accuracy did not improve from 0.97750
Epoch 39/128
113/113 [==============================] - 3s 27ms/step - loss: 0.0757 - accuracy: 0.9767 - val_loss: 0.1145 - val_accuracy: 0.9675

Epoch 00039: val_accuracy did not improve from 0.97750
Epoch 40/128
113/113 [==============================] - 3s 27ms/step - loss: 0.0875 - accuracy: 0.9729 - val_loss: 0.1144 - val_accuracy: 0.9675

Epoch 00040: val_accuracy did not improve from 0.97750

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
Epoch 41/128
113/113 [==============================] - 3s 27ms/step - loss: 0.0954 - accuracy: 0.9660 - val_loss: 0.1144 - val_accuracy: 0.9675

Epoch 00041: val_accuracy did not improve from 0.97750
Epoch 42/128
113/113 [==============================] - 3s 28ms/step - loss: 0.0857 - accuracy: 0.9720 - val_loss: 0.1144 - val

In [14]:
base_model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(filepath='model.h5', monitor='val_accuracy',
                                                       save_best_only=True, verbose=1)
earlyStop = tensorflow.keras.callbacks.EarlyStopping(patience=16, monitor='val_accuracy',
                                                    restore_best_weights=True, verbose=1)
reduceLR = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, 
                                                patience=4, verbose=1)

model.fit(x_train, y_train, batch_size=32, epochs=128, verbose=1,
          validation_data = (x_test, y_test),
          callbacks=[checkpoint, earlyStop, reduceLR])

Epoch 1/128
113/113 [==============================] - 15s 87ms/step - loss: 2.3494 - accuracy: 0.4378 - val_loss: 3.5493 - val_accuracy: 0.3075

Epoch 00001: val_accuracy improved from -inf to 0.30750, saving model to model.h5
Epoch 2/128
113/113 [==============================] - 9s 77ms/step - loss: 0.6798 - accuracy: 0.7584 - val_loss: 7.6563 - val_accuracy: 0.3725

Epoch 00002: val_accuracy improved from 0.30750 to 0.37250, saving model to model.h5
Epoch 3/128
113/113 [==============================] - 9s 77ms/step - loss: 0.4491 - accuracy: 0.8536 - val_loss: 3.5727 - val_accuracy: 0.2200

Epoch 00003: val_accuracy did not improve from 0.37250
Epoch 4/128
113/113 [==============================] - 9s 76ms/step - loss: 0.4893 - accuracy: 0.8461 - val_loss: 0.8891 - val_accuracy: 0.7725

Epoch 00004: val_accuracy improved from 0.37250 to 0.77250, saving model to model.h5
Epoch 5/128
113/113 [==============================] - 9s 76ms/step - loss: 0.5981 - accuracy: 0.8399 - val_loss


Epoch 00038: val_accuracy did not improve from 0.99750
Epoch 39/128
113/113 [==============================] - 9s 77ms/step - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.0097 - val_accuracy: 0.9975

Epoch 00039: val_accuracy did not improve from 0.99750

Epoch 00039: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.
Epoch 40/128
113/113 [==============================] - 9s 76ms/step - loss: 0.0037 - accuracy: 0.9986 - val_loss: 0.0094 - val_accuracy: 0.9975

Epoch 00040: val_accuracy did not improve from 0.99750
Epoch 41/128
113/113 [==============================] - 9s 76ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0095 - val_accuracy: 0.9975

Epoch 00041: val_accuracy did not improve from 0.99750
Epoch 42/128
113/113 [==============================] - 9s 76ms/step - loss: 0.0048 - accuracy: 0.9989 - val_loss: 0.0095 - val_accuracy: 0.9975

Epoch 00042: val_accuracy did not improve from 0.99750
Epoch 43/128
113/113 [==============================] - 9s 

In [15]:
y_pred = model.predict(x_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))

0.9975

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

array([[40,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 40,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 40,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 40,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 40,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 40,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 40,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0, 39,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 40,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 40]])